In [2]:
import os

import torch
from torchvision.models.resnet import resnet50
from torchvision.models.mobilenet import mobilenet_v2
from torchvision.models.squeezenet import squeezenet1_0, squeezenet1_1
from torchvision.models.shufflenetv2 import shufflenet_v2_x1_0

if os.path.basename(os.getcwd()) == 'analysis':
    os.chdir('..')
from models.gate_wrapped_module import compute_flop_cost_change
from models.custom_resnet import filter_mapping_from_default_resnet, custom_resnet_50

In [2]:
path = '/home/eli/Downloads/checkpoint.resnet50.2018-07-16-4310.pth.tar'
aaa = torch.load(path)

In [9]:
state_dict = aaa['state_dict']
state_dict = {k[7:]:v for k,v in state_dict.items()}
net = resnet50(pretrained=False)
net.load_state_dict(state_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [65]:
def conv_weights_analysis(weights):
    no_kernals = weights.sum(-1).sum(-1) == 0
#     init_filters = no_kernals.size(0), no_kernals.size(1)
    in_non_zero = (no_kernals.sum(0) > 0).sum().item()
    out_non_zero = (no_kernals.sum(1) > 0).sum().item()
    init_cost = weights.numel()
    final_cost  = in_non_zero*out_non_zero *weights.size(2) * weights.size(3)
    return init_cost, final_cost
    

In [74]:
downsample = 4
res1, res2  = conv_weights_analysis(net.conv1.weight.data)
orig_total = res1/4 
new_total = res2/4
for l in range(1,5):
    if l > 1:
        downsample = downsample * 2
    layer = getattr(net, 'layer' + str(l))
    res1, res2  = conv_weights_analysis(layer[0].downsample[0].weight.data)
    orig_total += res1/(downsample**2) 
    new_total += res2/(downsample**2) 
    for i in range(len(layer)):
        for j in range(1,4):
            res1, res2  = conv_weights_analysis(getattr(layer[i],'conv'+str(j)).weight.data)
            orig_total += res1/(downsample**2) 
            new_total += res2/(downsample**2) 
print(orig_total, new_total, 1-new_total/orig_total)
            

76848.0 27683.25 0.6397661617738913


In [3]:
m =mobilenet_v2(pretrained=False)

downsample = 4
res1, res2  = conv_weights_analysis(net.conv1.weight.data)
orig_total = res1/4 
new_total = res2/4
for l in range(1,5):
    if l > 1:
        downsample = downsample * 2
    layer = getattr(net, 'layer' + str(l))
    res1, res2  = conv_weights_analysis(layer[0].downsample[0].weight.data)
    orig_total += res1/(downsample**2) 
    new_total += res2/(downsample**2) 
    for i in range(len(layer)):
        for j in range(1,4):
            res1, res2  = conv_weights_analysis(getattr(layer[i],'conv'+str(j)).weight.data)
            orig_total += res1/(downsample**2) 
            new_total += res2/(downsample**2) 
print(orig_total, new_total, 1-new_total/orig_total)

In [17]:
mob =mobilenet_v2(pretrained=False)

In [37]:
downsample = 1
flops = 0
for m in mob.modules():
    if not isinstance(m, torch.nn.Conv2d):
        if isinstance(m, torch.nn.Linear):
            flops += m.in_features * m.out_features / (224**2)
    else:
        if m.stride[0] == 2:
            downsample = 2 * downsample
        flops += m.in_channels* m.out_channels* m.kernel_size[0]* m.kernel_size[1] / (m.groups * downsample**2)
print(flops, downsample)

5994.385204081633 32


In [38]:
76848 / 5994.385204081633 , 76848 / 5968.875

(12.8199969444195, 12.874787962555757)